In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from jppype import Mosaic, vscode_theme

from fundus_vessels_toolkit import AVLabel, FundusData
from fundus_vessels_toolkit.pipelines.avseg_to_tree import AVSegToTree

vscode_theme()

HTML(value="<style>\n        .cell-output-ipywidget-background {\n                background: transparent !imp…

#### Load image

In [3]:
IMG = "20060523_49681_0100_PP.png"
PATH = Path("/home/gaby/These/src/Fundus/RelatedWorks/AutoMorph/")


# Path to the raw fundus image
RAW_PATH = PATH / "images" / IMG

# Path to the OD segmentation
OD_PATH = PATH / "Results/M2/optic_disc_cup/raw" / IMG

# Path to the artery/vein segmentation
AV_PATH = PATH / "Results/M2/artery_vein/raw" / IMG

fundus = FundusData(fundus=RAW_PATH, od=OD_PATH, vessels=AV_PATH)

In [4]:
from jppype.utils.color import colormap_by_name

GRAPH_AV_COLORS = {
    AVLabel.BKG: "grey",
    AVLabel.ART: "red",
    AVLabel.VEI: "blue",
    AVLabel.BOTH: "purple",
    AVLabel.UNK: "green",
}
AV_COLORS = {
    1: "firebrick",
    2: "blue",
    3: "grey",
}


def GENERIC_CMAP(x):
    return ["grey", "red", "blue", "purple", "green", "orange"][x % 6]

#### Extract the geometrical and topological features of the arterial and venous trees

In [5]:
from fundus_vessels_toolkit.segment_to_graph.av_tree_parsing import propagate_av_labels

av2tree = AVSegToTree()

graph_ini = av2tree.to_vgraph(fundus, label_av=False, simplify=False)
graph_labelled = av2tree.assign_av_labels(graph_ini, fundus.av, propagate_labels=False)
graph_propagated = propagate_av_labels(graph_labelled, inplace=False)
graph_simplified = av2tree.simplify_av_graph(graph_propagated, inplace=False)

In [6]:
m = Mosaic(3, cols_titles=["Initial AV Label", "Propagated AV Label", "Simplified Graph"])

fundus.draw(view=m, labels_opacity=0.3)
m[0]["graph"] = graph_labelled.jppype_layer(edge_map=True, boundaries=True, edge_labels=True)
m[0]["graph"].edges_cmap = graph_labelled.branches_attr["av"].map(GRAPH_AV_COLORS).to_dict()
m[0]["graph"].nodes_cmap = graph_labelled.nodes_attr["av"].map(GRAPH_AV_COLORS).to_dict()

m[1]["graph"] = graph_propagated.jppype_layer(bspline=True)
m[1]["graph"].edges_cmap = graph_propagated.branches_attr["av"].map(GRAPH_AV_COLORS).to_dict()
m[1]["graph"].nodes_cmap = graph_propagated.nodes_attr["av"].map(GRAPH_AV_COLORS).to_dict()

m[2]["graph"] = graph_simplified.jppype_layer(bspline=True, edge_labels=True)
m[2]["graph"].edges_cmap = graph_simplified.branches_attr["av"].map(GRAPH_AV_COLORS).to_dict()
m[2]["graph"].nodes_cmap = graph_simplified.nodes_attr["av"].map(GRAPH_AV_COLORS).to_dict()

m

GridBox(children=(HTML(value='<h3 style="text-align: center;">Initial AV Label</h3>'), HTML(value='<h3 style="…

In [7]:
from fundus_vessels_toolkit.segment_to_graph.av_tree_parsing import build_line_digraph, resolve_digraph_to_vtree

graph = graph_simplified.copy()
graph, line_list, line_tips, line_p = build_line_digraph(graph, od_yx=fundus.od_center)
tree = resolve_digraph_to_vtree(graph.copy(), line_list, line_tips, line_p)
tree.branches_attr["subtree"] = tree.subtrees_branch_labels()

In [8]:
m = Mosaic(2)
fundus.draw(view=m, labels_opacity=0.1)

m[0]["graph"] = graph.jppype_layer(bspline=True, edge_labels=True)
m[0]["tangents"] = graph.geometric_data().jppype_branches_tips_tangents(scaling=2)
m[0]["graph"].edges_cmap = graph.branches_attr["av"].map(GRAPH_AV_COLORS).to_dict()

m[1]["graph"] = tree.jppype_layer(bspline=True, edge_labels=True, node_labels=True)
m[1]["tangents"] = tree.geometric_data().jppype_branches_tips_tangents(scaling=2)
m[1]["graph"].edges_cmap = tree.branches_attr["subtree"].map(GENERIC_CMAP).to_dict()
m[1]["graph"].edges_labels = tree.branches_attr["subtree"].to_dict()

m

GridBox(children=(View2D(linkedTransformGroup='2f7f720f3e1a48f087cb4ff0995577ec'), View2D(linkedTransformGroup…

In [9]:
build_line_digraph(graph_simplified.copy(), od_yx=fundus.od_center)
None

In [10]:
from fundus_vessels_toolkit.segment_to_graph.av_tree_parsing import inspect_digraph_solving

tables, g, t, lines = inspect_digraph_solving(fundus, graph_simplified)
tables

GridBox(children=(View2D(linkedTransformGroup='a6c13de5e9d14ee4b1848e0fa97659d2'), View2D(linkedTransformGroup…

BokehModel(combine_events=True, render_bundle={'docs_json': {'856a8ef7-75fa-441e-be3f-f271e6b0e283': {'version…